## Importing necessary modules

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# TensorFlow (already installed in Colab, but you can upgrade if needed)
!pip install -q --upgrade tensorflow

# imgaug (compatible with older NumPy)
!pip install -q imgaug==0.4.0

# Downgrade NumPy to avoid issues with imgaug (NumPy ≥ 2.0 breaks imgaug)
!pip install -q numpy==1.26.4

In [ ]:
import time
from tqdm import tqdm # Cool progress bar

import random
import numpy as np
import pandas as pd
import sys
import os # read and manipulate local files

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import cv2
import seaborn as sns

from PIL import Image

import tensorflow.keras as ks
import tensorflow as tf

from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K # F1-score metric

from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import   Lambda, Conv2D, MaxPool2D, UpSampling2D, BatchNormalization, Flatten
from tensorflow.keras.layers import  GlobalAveragePooling2D, Reshape, Multiply, Attention, add,Resizing,  Input, Dense
from tensorflow.keras.layers import Activation,AveragePooling2D, MaxPooling2D, Dropout, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model, Sequential

# hide wornings
import warnings
warnings.filterwarnings('ignore')

# from keras import ops# For deeplab3

# import keras_cv # basnet

from imgaug import augmenters as iaa ## for augmentation

## Defining paths

In [ ]:
model_name = 'Basic_Unet_2500'

################################################################################################
## SETTING THE PATHS
PATH_TO_CODE ='/content/drive/MyDrive/TRIAL_v1/segmentation_task'
# PATH_TO_CODE =''
sys.path.append(PATH_TO_CODE)

# DATASET_PATH = r'C:\Users\ASUS\Desktop\segmentation & cropping\segmentation_task\segmentation_data43_resized_cropped_split'
DATASET_PATH = r'/content/drive/MyDrive/TRIAL_v1/segmentation_task/segmentation_data43_resized_cropped_split'
# DATASET_PATH = r'segmentation_data43_resized_cropped_split'
DATASET_PATH_TRAIN = os.path.join(DATASET_PATH, 'train')
DATASET_PATH_TEST = os.path.join(DATASET_PATH, 'test')


PATH_TO_SAVE_RESULT = os.path.join(PATH_TO_CODE, 'saved_outputs_segmentation_AUC', model_name)
PATH_BEST_SAVE_WEIGHT = os.path.join(PATH_TO_SAVE_RESULT,'saved_weights')
PATH_TO_SAVE_MODEL = os.path.join(PATH_TO_SAVE_RESULT, 'saved_models')
PATH_SAVE_HISTORY = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_training_history.csv')
PATH_SAVE_TIME = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_training_time.csv')
PATH_SAVE_AUGMENT_SAMPLE = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_augmented_sample.pdf')
PATH_SAVE_PIXEL_PERCENTAGE_PLOT = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_pixel_percentage.pdf')
# PATH_TO_SAVE_TUNER = os.path.join(PATH_TO_RESULT, 'saved_tuner_model')
################################################################################################

if not os.path.exists(PATH_TO_SAVE_RESULT):
 # If it does not exist, create it
    os.makedirs(PATH_TO_SAVE_RESULT)
################################################################################################
IMG_HEIGHT = 256
IMG_WIDTH = 256
ORIGINAL_IMAGE_SIZE = (IMG_HEIGHT, IMG_WIDTH)
COLOR_CHANNEL = 3

RESIZE_SHAPE = ORIGINAL_IMAGE_SIZE #(128, 128) #ORIGINAL_IMAGE_SIZE#(128, 128)# # #ORIGINAL_IMAGE_SIZE# (128, 128)#
MODEL_INPUT_SIZE = (RESIZE_SHAPE[0], RESIZE_SHAPE[1], COLOR_CHANNEL)

VALIDATION_SPLIT= 0.2
NUM_CLASSES = 2 # Disease and not disease

BATCH_SIZE = 10
EPOCHS = 70

TOTAL_DATA = 2500
################################################################################################

################################################################################################

SAVE_RESULTS = True
SHOW_RESULTS = True

################################################################################################
# Setting the seed
SEED  = 123
RNG = np.random.default_rng(SEED) # Random number generator
tf.random.set_seed(SEED)

################################################################################################
# Checkpoint parameters val_binary_io_u
SCORE_TO_MONITOR = 'val_binary_io_u' # Score that checkpoints monitor during training
SCORE_OBJECTIVE  = 'max'          # 'max' or 'min', specifies whether the objective is to maximize the score or minimize it.
PATIENCE_LR_REDUCE = 5
MIN_LR = 1e-8
REDUCTION_FACTOR = 0.5            # Factor which lr will be reduced with at plateau
COOLDOWN_EPOCHS  = 2 #cooldown: Integer. Number of epochs to wait before resuming normal operation after the learning rate has been reduced.

INITIAL_LR = 0.001

## Defining performance metrics

Official segmentation metrics by keras: https://ks.io/api/metrics/segmentation_metrics/

In [ ]:
def f1_score(y_true, y_pred): # Dice coefficient
    """
    Calculate the F1 score, the harmonic mean of precision and recall, for binary classification.

    Args:
        y_true (Tensor): True binary labels.
        y_pred (Tensor): Predicted probabilities.

    Returns:
        float32: F1 score as a scalar.
    """
    # True Positives: round product of y_true and y_pred
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    # Actual Positives: round y_true
    P = K.sum(K.round(K.clip(y_true, 0, 1)))
    # Recall: TP / Actual Positives
    recall = TP / (P + K.epsilon())

    # Predicted Positives: round y_pred
    Pred_P = K.sum(K.round(K.clip(y_pred, 0, 1)))
    # Precision: TP / Predicted Positives
    precision = TP / (Pred_P + K.epsilon())

    # F1 Score: harmonic mean of precision and recall
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

# source: https://www.tensorflow.org/api_docs/python/tf/keras/metrics/BinaryIoU
iou_score_binary = tf.keras.metrics.BinaryIoU(target_class_ids=[0, 1], threshold=0.5)

def accuracy_score(y_true, y_pred):
    """
    Calculate accuracy score between two binary masks.
    """
    correct = np.sum(y_true == y_pred)  # Count correct predictions
    total = y_true.size  # Total number of pixels
    return correct / total  # Accuracy calculation

def precision_score(groundtruth_mask, pred_mask):
    """
    Calculate precision score between two binary masks.
    """
    intersect = np.sum(pred_mask * groundtruth_mask)  # Calculate intersection
    total_pixel_pred = np.sum(pred_mask)  # Sum of predicted positives
    return np.mean(intersect / total_pixel_pred)  # Precision calculation

def recall_score(groundtruth_mask, pred_mask):
    """
    Calculate recall score between two binary masks.
    """
    intersect = np.sum(pred_mask * groundtruth_mask)  # Calculate intersection
    total_pixel_truth = np.sum(groundtruth_mask)  # Sum of actual positives
    return np.mean(intersect / total_pixel_truth)  # Recall calculation

## Reading the data

In [ ]:
# Function to read images from a directory using Pillow
def read_images_from_directory(directory_path):
    images = []
    for filename in sorted(os.listdir(directory_path)):
        # filename= filename.lower()

        if filename.endswith(('.png', '.jpg', '.JPG' )):  # Check for image file extensions
            img_path = os.path.join(directory_path, filename)
            img = Image.open(img_path)
            img_array = np.array(img)  # Convert the image to a numpy array if needed
            if img_array is not None:
                images.append(img_array)
    return images

# Paths to masks and original images within the dataset
masks_path_train = os.path.join(DATASET_PATH_TRAIN, 'data43_masks_binarised')
originals_path_train = os.path.join(DATASET_PATH_TRAIN, 'data43_original')

# Read images into variables
x_train = read_images_from_directory(originals_path_train)  # Original images
y_train = read_images_from_directory(masks_path_train)  # Masks

# Paths to masks and original images within the test dataset
masks_path_test = os.path.join(DATASET_PATH_TEST, 'data43_masks_binarised')
originals_path_test = os.path.join(DATASET_PATH_TEST, 'data43_original')

# Read images into variables
x_test = read_images_from_directory(originals_path_test)  # Original images
y_test = read_images_from_directory(masks_path_test)  # Masks

# Now x_train contains original images, and y_train contains mask images
print(f"Loaded {len(x_train)} original images into x_train")
print(f"Loaded {len(y_train)} mask images into y_train")

# Similarly, for test images
print(f"Loaded {len(x_test)} original images into x_test")
print(f"Loaded {len(y_test)} mask images into y_test")

Loaded 616 original images into x_train
Loaded 616 mask images into y_train
Loaded 23 original images into x_test
Loaded 23 mask images into y_test


## Augmentations

In [ ]:
seq = iaa.Sequential([
    iaa.Fliplr(0.5),  # horizontally flip 50% of the images
    iaa.Flipud(0.2),  # vertically flip 20% of the images
    iaa.Affine(
        scale={"x": (0.8, 1.1), "y": (0.8, 1.1)},  # zoom in or out (80-120%)
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},  # width & height shift (-20 to +20%)
        rotate=(-15, 15),  # rotation (-45 to 45 degrees)
        shear=(-16, 16)  # shear (-16 to 16 degrees)
    ),
    iaa.ElasticTransformation(alpha=(0, 2.0), sigma=0.25),  # apply elastic deformations
    iaa.GaussianBlur(sigma=(0, 2.0))  # apply Gaussian blur
])

In [ ]:
# Function to augment a batch of images and masks
def augment_batch(images, masks, seq, batch_size):
    augmented_images = []
    augmented_masks = []
    while len(augmented_images) < batch_size:
        aug_images, aug_masks = seq(images=images, segmentation_maps=np.expand_dims(masks, axis=-1))
        augmented_images.extend(aug_images)
        augmented_masks.extend(aug_masks)

        if len(augmented_images) >= batch_size:
            break

    # Ensure we only take as many as we need to reach the desired batch_size
    augmented_images = augmented_images[:batch_size]
    augmented_masks = augmented_masks[:batch_size]

    return np.array(augmented_images), np.squeeze(np.array(augmented_masks), axis=-1)

In [ ]:
num_augmented_images_needed = TOTAL_DATA  - len(x_train)  # Calculate how many augmented images we need
if num_augmented_images_needed > 0:
    augmented_x, augmented_y = augment_batch(x_train, y_train, seq, num_augmented_images_needed)
    # Concatenate the original and augmented datasets
    combined_x_train = np.concatenate((x_train, augmented_x), axis=0)
    combined_y_train = np.concatenate((y_train, augmented_y), axis=0)
else:
    combined_x_train = x_train
    combined_y_train = y_train

In [ ]:
x_train = combined_x_train
y_train = combined_y_train

## Preprocessing DO NOT RESIZE THE IMAGE-- IT GIVES FINE LINES IN THE OUTPUT

In [ ]:
# # # Resizing
# x_train = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in x_train]
# y_train = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in y_train]

# x_test = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in x_test]
# y_test = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in y_test]

# Convert the list to a NumPy array
x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = np.array(x_test)
y_test = np.array(y_test)

# Expand mask dimensions to (batch, 256, 256, 1)
y_train = np.expand_dims(y_train, axis=-1)
y_test = np.expand_dims(y_test, axis=-1)


# Normalizing input between [0,1]
x_train = x_train.astype("float32")/ np.max(x_train)
x_test  = x_test.astype("float32")/np.max(x_test)

y_train = y_train.astype("float32")/ np.max(y_train)
y_test  = y_test.astype("float32")/np.max(y_test)

# Now x_train contains original images, and y_train contains mask images
print(f"Loaded {len(x_train)} original images into x_train")
print(f"Loaded {len(y_train)} mask images into y_train")

# Similarly, for test images
print(f"Loaded {len(x_test)} original images into x_test")
print(f"Loaded {len(y_test)} mask images into y_test")

Loaded 2500 original images into x_train
Loaded 2500 mask images into y_train
Loaded 23 original images into x_test
Loaded 23 mask images into y_test


## Essential functions

In [ ]:
def fit_and_save_best_model(model_name, model, epochs=EPOCHS):
    ##########################################################################################################
    saved_best_model_name = 'best_'+model_name+'.h5'

    model_saving_path = os.path.join(PATH_TO_SAVE_MODEL, saved_best_model_name)
    # Set up a model checkpoint to save the best model during training
    best_model_callback= ModelCheckpoint(model_saving_path,
                                          monitor=SCORE_TO_MONITOR,
                                          save_best_only=True,
                                          mode=SCORE_OBJECTIVE,
                                          verbose=1)

    reduce_LR = ReduceLROnPlateau(
        factor=REDUCTION_FACTOR,      # Factor by which the learning rate will be reduced. new_lr = lr * factor
        patience=PATIENCE_LR_REDUCE, # original was 5      # Number of epochs with no improvement after which learning rate will be reduced.
        verbose=1,       # int. 0: quiet, 1: update messages.
        min_lr=MIN_LR   # Lower bound on the learning rate.
    )

    my_callbacks = [best_model_callback , reduce_LR]
    ##########################################################################################################

    start_time = time.time()

    # Fitting the model
    train_history = model.fit(
        x_train,
        y_train,
        epochs= epochs,
        batch_size=BATCH_SIZE,
        validation_split= VALIDATION_SPLIT,
        callbacks=my_callbacks,
    )
    total_time = time.time() - start_time

    return saved_best_model_name, train_history, total_time

## Models
### Models worked
- Model01: Basic basic_unet
- Model02: unet_vgg16 (my own custom)
- Model03: TransUNet
- Model04: DeepLab-v3Plus
- Model05: ResUNet
- Model06: UNet_with_attention
- Model07: Multi-resUnet
- Model08: Inception_resnetV2 (my own custom)



### Basic basic_unet

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    x = Activation('relu')(x)

    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(x)
    x = Activation('relu')(x)

    return x

def get_basic_unet(n_filters = 16):
    """
    Version of U-Net with dropout and size preservation (padding= 'same')--from class note
    """
    input_img = Input(shape=MODEL_INPUT_SIZE)
    # Contracting Path
    c1 = conv2d_block(input_img, n_filters * 1, kernel_size=3)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = conv2d_block(p1, n_filters * 2, kernel_size=3)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = conv2d_block(p2, n_filters * 4, kernel_size=3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = conv2d_block(p3, n_filters * 8, kernel_size=3)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = conv2d_block(p4, n_filters * 16, kernel_size=3)

    # Expansive Path
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides=(2, 2), padding='same')(c5)
    u6 = Concatenate()([u6, c4])
    c6 = conv2d_block(u6, n_filters * 8, kernel_size=3)

    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides=(2, 2), padding='same')(c6)
    u7 = Concatenate()([u7, c3])
    c7 = conv2d_block(u7, n_filters * 4, kernel_size=3)

    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides=(2, 2), padding='same')(c7)
    u8 = Concatenate()([u8, c2])
    c8 = conv2d_block(u8, n_filters * 2, kernel_size=3)

    u9 = Conv2DTranspose(n_filters, (3, 3), strides=(2, 2), padding='same')(c8)
    u9 = Concatenate()([u9, c1])
    c9 = conv2d_block(u9, n_filters, kernel_size=3)

    outputs = Conv2D(NUM_CLASSES-1, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input_img], outputs=[outputs])

    optimizer = Adam(learning_rate=INITIAL_LR)
    model.compile(loss='binary_crossentropy',
                  metrics=['accuracy', f1_score, iou_score_binary],
                  optimizer=optimizer)

    return model

## Train and Results

In [ ]:
model= get_basic_unet() ## Model1
# model= get_unet_vgg16() ## Model2
# model= get_TransUNet() ## Model3
# model= get_DeepLabV3Plus() ## Model4
# model = get_ResUNet_v3() ## Model5
# model = UNet_with_attention() ## Model6
# model = get_multiresunet() ## Model7
# model = get_unet_InceptionResNetV2() ## Model8

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │        448 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 256, 256,  │          0 │ conv2d[0][0]      │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │      2,320 │ activation[0][0]  │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 256, 256,  │          0 │ conv2d_1[0][0]    │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │      4,640 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 128, 128,  │          0 │ conv2d_2[0][0]    │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │      9,248 │ activation_2[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 128, 128,  │          0 │ conv2d_3[0][0]    │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │     18,496 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 64, 64,    │          0 │ conv2d_4[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 64, 64,    │     36,928 │ activation_4[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_5        │ (None, 64, 64,    │          0 │ conv2d_5[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 32, 32,    │          0 │ activation_5[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 32, 32,    │     73,856 │ max_pooling2d_2[

 Total params: 2,158,705 (8.23 MB)

 Trainable params: 2,158,705 (8.23 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# training the model and saving the best model as a check point
best_model_name, train_history, total_time = fit_and_save_best_model(model_name, model)

Epoch 1/70
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9572 - binary_io_u: 0.4881 - f1_score: 0.0231 - loss: 0.2013
Epoch 1: val_binary_io_u improved from -inf to 0.65655, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Basic_Unet_2500/saved_models/best_Basic_Unet_2500.h5


200/200 ━━━━━━━━━━━━━━━━━━━━ 31s 70ms/step - accuracy: 0.9573 - binary_io_u: 0.4887 - f1_score: 0.0245 - loss: 0.2006 - val_accuracy: 0.9740 - val_binary_io_u: 0.6566 - val_f1_score: 0.4509 - val_loss: 0.0781 - learning_rate: 0.0010
Epoch 2/70
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9760 - binary_io_u: 0.7365 - f1_score: 0.6441 - loss: 0.0733
Epoch 2: val_binary_io_u improved from 0.65655 to 0.76932, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Basic_Unet_2500/saved_models/best_Basic_Unet_2500.h5


200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9761 - binary_io_u: 0.7366 - f1_score: 0.6444 - loss: 0.0732 - val_accuracy: 0.9819 - val_binary_io_u: 0.7693 - val_f1_score: 0.6624 - val_loss: 0.0560 - learning_rate: 0.0010
Epoch 3/70
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9792 - binary_io_u: 0.7716 - f1_score: 0.7037 - loss: 0.0630
Epoch 3: val_binary_io_u did not improve from 0.76932
200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.9792 - binary_io_u: 0.7715 - f1_score: 0.7037 - loss: 0.0630 - val_accuracy: 0.9820 - val_binary_io_u: 0.7674 - val_f1_score: 0.6565 - val_loss: 0.0551 - learning_rate: 0.0010
Epoch 4/70
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9820 - binary_io_u: 0.7992 - f1_score: 0.7473 - loss: 0.0537
Epoch 4: val_binary_io_u improved from 0.76932 to 0.78226, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Basic_Unet_2500/saved_models/best_Basic_Unet_2500.h5


200/200 ━━━━━━━━━━━━━━━━━━━━ 10s 48ms/step - accuracy: 0.9820 - binary_io_u: 0.7992 - f1_score: 0.7472 - loss: 0.0537 - val_accuracy: 0.9830 - val_binary_io_u: 0.7823 - val_f1_score: 0.6746 - val_loss: 0.0515 - learning_rate: 0.0010
Epoch 5/70
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9824 - binary_io_u: 0.8043 - f1_score: 0.7553 - loss: 0.0515
Epoch 5: val_binary_io_u improved from 0.78226 to 0.79142, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Basic_Unet_2500/saved_models/best_Basic_Unet_2500.h5


200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9824 - binary_io_u: 0.8042 - f1_score: 0.7553 - loss: 0.0515 - val_accuracy: 0.9835 - val_binary_io_u: 0.7914 - val_f1_score: 0.6923 - val_loss: 0.0492 - learning_rate: 0.0010
Epoch 6/70
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9828 - binary_io_u: 0.8085 - f1_score: 0.7619 - loss: 0.0500
Epoch 6: val_binary_io_u did not improve from 0.79142
200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.9828 - binary_io_u: 0.8084 - f1_score: 0.7619 - loss: 0.0500 - val_accuracy: 0.9836 - val_binary_io_u: 0.7910 - val_f1_score: 0.6883 - val_loss: 0.0488 - learning_rate: 0.0010
Epoch 7/70
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9835 - binary_io_u: 0.8152 - f1_score: 0.7708 - loss: 0.0477
Epoch 7: val_binary_io_u improved from 0.79142 to 0.80088, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Basic_Unet_2500/saved_models/best_Basic_Unet_2500.h5


...

200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9905 - binary_io_u: 0.8894 - f1_score: 0.8708 - loss: 0.0249 - val_accuracy: 0.9882 - val_binary_io_u: 0.8502 - val_f1_score: 0.7854 - val_loss: 0.0352 - learning_rate: 5.0000e-04
Epoch 39/70
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9909 - binary_io_u: 0.8941 - f1_score: 0.8769 - loss: 0.0235
Epoch 39: val_binary_io_u improved from 0.85023 to 0.85044, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Basic_Unet_2500/saved_models/best_Basic_Unet_2500.h5


200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9909 - binary_io_u: 0.8940 - f1_score: 0.8768 - loss: 0.0235 - val_accuracy: 0.9883 - val_binary_io_u: 0.8504 - val_f1_score: 0.7853 - val_loss: 0.0367 - learning_rate: 5.0000e-04
Epoch 40/70
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9912 - binary_io_u: 0.8975 - f1_score: 0.8812 - loss: 0.0226
Epoch 40: val_binary_io_u improved from 0.85044 to 0.85265, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Basic_Unet_2500/saved_models/best_Basic_Unet_2500.h5


200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9912 - binary_io_u: 0.8974 - f1_score: 0.8812 - loss: 0.0226 - val_accuracy: 0.9883 - val_binary_io_u: 0.8526 - val_f1_score: 0.7913 - val_loss: 0.0365 - learning_rate: 5.0000e-04
Epoch 41/70
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9914 - binary_io_u: 0.8998 - f1_score: 0.8842 - loss: 0.0219
Epoch 41: val_binary_io_u did not improve from 0.85265
200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.9914 - binary_io_u: 0.8998 - f1_score: 0.8841 - loss: 0.0219 - val_accuracy: 0.9882 - val_binary_io_u: 0.8519 - val_f1_score: 0.7900 - val_loss: 0.0374 - learning_rate: 5.0000e-04
Epoch 42/70
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9915 - binary_io_u: 0.9009 - f1_score: 0.8855 - loss: 0.0215
Epoch 42: val_binary_io_u did not improve from 0.85265

Epoch 42: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.9915 - binary_io_u: 0.9009

200/200 ━━━━━━━━━━━━━━━━━━━━ 10s 47ms/step - accuracy: 0.9921 - binary_io_u: 0.9080 - f1_score: 0.8952 - loss: 0.0194 - val_accuracy: 0.9886 - val_binary_io_u: 0.8590 - val_f1_score: 0.8012 - val_loss: 0.0387 - learning_rate: 1.2500e-04
Epoch 49/70
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9923 - binary_io_u: 0.9096 - f1_score: 0.8972 - loss: 0.0190
Epoch 49: val_binary_io_u improved from 0.85897 to 0.85944, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Basic_Unet_2500/saved_models/best_Basic_Unet_2500.h5


200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.9923 - binary_io_u: 0.9095 - f1_score: 0.8972 - loss: 0.0190 - val_accuracy: 0.9886 - val_binary_io_u: 0.8594 - val_f1_score: 0.8029 - val_loss: 0.0403 - learning_rate: 1.2500e-04
Epoch 50/70
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9925 - binary_io_u: 0.9124 - f1_score: 0.9009 - loss: 0.0184
Epoch 50: val_binary_io_u did not improve from 0.85944
200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.9925 - binary_io_u: 0.9124 - f1_score: 0.9008 - loss: 0.0184 - val_accuracy: 0.9886 - val_binary_io_u: 0.8593 - val_f1_score: 0.8027 - val_loss: 0.0411 - learning_rate: 1.2500e-04
Epoch 51/70
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9927 - binary_io_u: 0.9146 - f1_score: 0.9035 - loss: 0.0180
Epoch 51: val_binary_io_u did not improve from 0.85944
200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 42ms/step - accuracy: 0.9927 - binary_io_u: 0.9145 - f1_score: 0.9035 - loss: 0.0180 - val_accuracy: 0.9886 - val_binary_io_u: 0

## Loading the best model

In [ ]:
##Now, load the best model
best_model = ks.models.load_model(os.path.join(PATH_TO_SAVE_MODEL, best_model_name),
                                  custom_objects={'f1_score': f1_score,
                                                  'binary_io_u':iou_score_binary})

# best_model =model

## Visualize model predictions

In [ ]:
# Get predictions from the model
predictions = best_model.predict(x_test, verbose=1)

USER_DETERMINED_THRESHOLD = 0.5
thresholded_predictions  = (predictions  >= USER_DETERMINED_THRESHOLD)
# Removing the color channel
thresholded_predictions_without_color_channel = np.squeeze(thresholded_predictions, axis=-1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


# AUC

In [ ]:
# Add this section *after* prediction is made with the best model
# i.e., after: predictions = best_model.predict(x_test, verbose=1)

from sklearn.metrics import roc_auc_score

# Flatten masks for AUC computation
y_true_flat = y_test.flatten()
y_score_flat = predictions.flatten()  # Predicted probabilities

In [ ]:
# Bootstrapping AUC
n_iterations = 100
rng = np.random.default_rng(SEED)
bootstrap_aucs = []

for _ in range(n_iterations):
    indices = rng.integers(0, len(y_true_flat), len(y_true_flat))
    y_true_sample = y_true_flat[indices]
    y_score_sample = y_score_flat[indices]

    # Ensure both classes are present in the sample
    if len(np.unique(y_true_sample)) < 2:
        continue

    auc = roc_auc_score(y_true_sample, y_score_sample)
    bootstrap_aucs.append(auc)

In [ ]:
# Calculate statistics
mean_auc = np.mean(bootstrap_aucs)
std_auc = np.std(bootstrap_aucs)
lower_ci = np.percentile(bootstrap_aucs, 2.5)
upper_ci = np.percentile(bootstrap_aucs, 97.5)

# Print AUC statistics
print(f"Bootstrapped AUC: Mean={mean_auc:.4f}, Std={std_auc:.4f}, 95% CI=({lower_ci:.4f}, {upper_ci:.4f})")

# Optional saving
if SAVE_RESULTS:
    auc_df = pd.DataFrame({'AUC Scores': bootstrap_aucs})
    auc_df.to_csv(os.path.join(PATH_TO_SAVE_RESULT, f"{model_name}_bootstrapped_auc.csv"), index=False)

Bootstrapped AUC: Mean=0.9850, Std=0.0003, 95% CI=(0.9844, 0.9855)
